In [ ]:
#!pip install plotly
# Instalação da biblioteca plotly

## Análise de Impactos Covid-19 (Estudo de Caso)

A primeira onda de covid-19 impactou a economia global, pois o mundo nunca esteve pronto para a pandemia. Isso resultou em aumento de casos, aumento de mortes, aumento do desemprego e aumento da pobreza, resultando em uma desaceleração econômica. Aqui, você é obrigado a analisar a disseminação dos casos de Covid-19 e todos os impactos da covid-19 na economia.

O conjunto de dados que estamos usando para analisar os impactos da covid-19 é baixado do Kaggle. Contém dados sobre:

* o código do país
* nome de todos os países
* data do registro
* Índice de desenvolvimento humano de todos os países
* Casos diários de covid-19
* Mortes diárias por covid-19
* índice de rigor dos países
* a população dos países
* PIB per capita dos países

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('dados/transformed_data.csv')
data2 = pd.read_csv('dados/raw_data.csv')
data.head()

,CODE,COUNTRY,DATE,HDI,TC,TD,STI,POP,GDPCAP
0,AFG,Afghanistan,2019-12-31,0.498,0.0,0.0,0.0,17.477233,7.497754
1,AFG,Afghanistan,2020-01-01,0.498,0.0,0.0,0.0,17.477233,7.497754
2,AFG,Afghanistan,2020-01-02,0.498,0.0,0.0,0.0,17.477233,7.497754
3,AFG,Afghanistan,2020-01-03,0.498,0.0,0.0,0.0,17.477233,7.497754
4,AFG,Afghanistan,2020-01-04,0.498,0.0,0.0,0.0,17.477233,7.497754


In [3]:
data2.head()

,iso_code,location,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494


In [4]:
data['COUNTRY'].value_counts().mode()

0    294
dtype: int64

In [6]:
#Agregando os dados

code = data["CODE"].unique().tolist() #Unificou os códigos do paíse e transformou e lista
country = data["COUNTRY"].unique().tolist() #Unificou os países e transformou em lista
hdi = [] # criando uma lista vazia de Human Development Index
tc = [] # criando uma lista vazia de total_cases
td = [] # criando uma lista vazia de total_deaths
sti = [] # criando uma lista vazia de Sexually transmitted infections
population = data["POP"].unique().tolist()#Unificou os valores totais da população e transformou e lista
gdp = [] 

for i in country:
    hdi.append((data.loc[data["COUNTRY"] == i, "HDI"]).sum()/294)
    tc.append((data2.loc[data2["location"] == i, "total_cases"]).sum())
    td.append((data2.loc[data2["location"] == i, "total_deaths"]).sum())
    sti.append((data.loc[data["COUNTRY"] == i, "STI"]).sum()/294)
    population.append((data2.loc[data2["location"] == i, "population"]).sum()/294)

aggregated_data = pd.DataFrame(list(zip(code, country, hdi, tc, td, sti, population)), 
                               columns = ["Country Code", "Country", "HDI", 
                                          "Total Cases", "Total Deaths", 
                                          "Stringency Index", "Population"])


In [7]:
aggregated_data

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
0,AFG,Afghanistan,0.498000,5126433.0,165875.0,3.049673,17.477233
1,ALB,Albania,0.600765,1071951.0,31056.0,3.005624,14.872537
2,DZA,Algeria,0.754000,4893999.0,206429.0,3.195168,17.596309
3,AND,Andorra,0.659551,223576.0,9850.0,2.677654,11.254996
4,AGO,Angola,0.418952,304005.0,11820.0,2.965560,17.307957
...,...,...,...,...,...,...,...
205,VEN,Venezuela,0.566867,4839834.0,40840.0,3.235752,17.163165
206,VNM,Vietnam,0.694000,122618.0,2318.0,3.710868,18.393706
207,YEM,Yemen,0.296721,228925.0,64304.0,2.369568,17.210890
208,ZMB,Zambia,0.430000,1129913.0,26475.0,2.645104,16.726989


In [12]:
# Seleção dos países para amostra para fazer o estudo, devido a complexidade de coletar o PBI de cada País.
data = aggregated_data.sort_values(by=["Total Cases"], ascending=False)
data.head()

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
200,USA,United States,0.92400,746014098.0,26477574.0,3.350949,19.617637
27,BRA,Brazil,0.75900,425704517.0,14340567.0,3.136028,19.174732
90,IND,India,0.64000,407771615.0,7247327.0,3.610552,21.045353
157,RUS,Russia,0.81600,132888951.0,2131571.0,3.380088,18.798668
150,PER,Peru,0.59949,74882695.0,3020038.0,3.430126,17.311165


In [14]:
data = data.head(10)
data

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
200,USA,United States,0.924000,746014098.0,26477574.0,3.350949,19.617637
27,BRA,Brazil,0.759000,425704517.0,14340567.0,3.136028,19.174732
90,IND,India,0.640000,407771615.0,7247327.0,3.610552,21.045353
157,RUS,Russia,0.816000,132888951.0,2131571.0,3.380088,18.798668
150,PER,Peru,0.599490,74882695.0,3020038.0,3.430126,17.311165
125,MEX,Mexico,0.774000,74347548.0,7295850.0,3.019289,18.674802
178,ESP,Spain,0.887969,73717676.0,5510624.0,3.393922,17.660427
175,ZAF,South Africa,0.608653,63027659.0,1357682.0,3.364333,17.898266
42,COL,Colombia,0.581847,60543682.0,1936134.0,3.357923,17.745037
199,GBR,United Kingdom,0.922000,59475032.0,7249573.0,3.353883,18.033340


In [16]:
data["GDP Before Covid"] = [65279.53, 8897.49, 2100.75, 
                            11497.65, 7027.61, 9946.03, 
                            29564.74, 6001.40, 6424.98, 42354.41]
data["GDP During Covid"] = [63543.58, 6796.84, 1900.71, 
                            10126.72, 6126.87, 8346.70, 
                            27057.16, 5090.72, 5332.77, 40284.64]
data.head(10)

<ipython-input-16-2fbcc8b59241>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["GDP Before Covid"] = [65279.53, 8897.49, 2100.75,
<ipython-input-16-2fbcc8b59241>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["GDP During Covid"] = [63543.58, 6796.84, 1900.71,


,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population,GDP Before Covid,GDP During Covid
200,USA,United States,0.924000,746014098.0,26477574.0,3.350949,19.617637,65279.53,63543.58
27,BRA,Brazil,0.759000,425704517.0,14340567.0,3.136028,19.174732,8897.49,6796.84
90,IND,India,0.640000,407771615.0,7247327.0,3.610552,21.045353,2100.75,1900.71
157,RUS,Russia,0.816000,132888951.0,2131571.0,3.380088,18.798668,11497.65,10126.72
150,PER,Peru,0.599490,74882695.0,3020038.0,3.430126,17.311165,7027.61,6126.87
125,MEX,Mexico,0.774000,74347548.0,7295850.0,3.019289,18.674802,9946.03,8346.70
178,ESP,Spain,0.887969,73717676.0,5510624.0,3.393922,17.660427,29564.74,27057.16
175,ZAF,South Africa,0.608653,63027659.0,1357682.0,3.364333,17.898266,6001.40,5090.72
42,COL,Colombia,0.581847,60543682.0,1936134.0,3.357923,17.745037,6424.98,5332.77
199,GBR,United Kingdom,0.922000,59475032.0,7249573.0,3.353883,18.033340,42354.41,40284.64


In [19]:
#Plotagem dos dados para entendimento dos países com maiores casos de covid-19 - DataSet: 31/12/2019 - 10/10/2020
figure = px.bar(data, y='Total Cases', x='Country', title="Países com maior número de caso de Covid (DataSet: 31/12/2019 - 10/10/2020)")
figure.show()

In [21]:
figure2  = px.bar(data,y='Total Deaths', x = 'Country', title = 'Países com o Maior Número de Mortes (DataSet: 31/12/2019 - 10/10/2020)')
figure2.show()

In [22]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["Total Cases"],
    name='Total Cases',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["Total Deaths"],
    name='Total Deaths',
    marker_color='lightsalmon'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [23]:
# Percentage of Total Cases and Deaths
cases = data["Total Cases"].sum()
deceased = data["Total Deaths"].sum()

labels = ["Total Cases", "Total Deaths"]
values = [cases, deceased]

fig = px.pie(data, values=values, names=labels, 
             title='Porcentagem do Total Casos e Mortes por COVID-19', hole=0.5)
fig.show()

In [25]:
#Cálculo da Taxa de mortalidade (death_rate)
death_rate = (data["Total Deaths"].sum() / data["Total Cases"].sum()) * 100
print(f"Death Rate = {death_rate:.2f}% ")

Death Rate = 3.61% 


### O Stringency Index é uma medida composta com base em nove indicadores de resposta ( fechamento de escolas; fechamentos de locais de trabalho; cancelamento de eventos públicos; restrições a reuniões públicas; fechamento de transporte público; requisitos de permanência em casa; campanhas de informação pública; restrições aos movimentos internos; e controles de viagens internacionais) redimensionados para um valor de 0 a 100 (100 = mais rigoroso).

In [26]:
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'], 
             color='Stringency Index', height=400, 
             title= "Stringency Index during Covid-19")
fig.show()

In [27]:
# Impacto na Economia antes dao covid-19
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'], #hover_data: insere um label das demais informações no grafico
             color='GDP Before Covid', height=400, 
             title="GDP Per Capita Before Covid-19")
fig.show()

In [28]:
 # Impacto na Economia durante a covid-19
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'], #hover_data: insere um label das demais informações no grafico
             color='GDP During Covid', height=400, 
             title="GDP Per Capita During Covid-19")
fig.show()

In [36]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["GDP Before Covid"],
    name='GDP Per Capita Before Covid-19',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["GDP During Covid"],
    name='GDP Per Capita During Covid-19',
    marker_color='lightsalmon'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [38]:
fig = px.bar(data,x='Country', y = 'Total Cases',
            hover_data = ['Population', 'Total Deaths'],
            color='HDI', height=400,
            title = 'Human Development Index during Covid-19')
fig.show()

### Conclusão

Nesta tarefa, estudamos a disseminação da covid-19 entre os países e seu impacto na economia global. Vimos que o surto de covid-19 resultou no maior número de casos e mortes de covid-19 nos Estados Unidos. Uma das principais razões por trás disso é o Stringency Index nos Estados Unidos. É comparativamente baixo de acordo com a população. Também analisamos como o PIB per capita de cada país foi afetado durante o surto de covid-19.